## Forecasting Auckland’s Monthly House Price Index (2018–2025): Transparent Time-Series Baselines with Causal Exogenous Screening

### Student ID: 24251155

### Purpose:
  - Build a transparent pipeline for forecasting log(HPI) at horizons h={1,3,6,12}
  - Champions chosen by rolling-origin CV (2018-01→2024-08), audited on TEST (targets 2024-09→2025-08)
  - Baselines: Seasonal-naïve, ETS; Models: ARIMA, ARIMAX (with 4 pre-registered drivers)
  - Causal imputation happens IN-MEMORY here; we also export the exact imputed panel for audit.

### Analysis policy abstract:
  - Main ARIMAX drivers (lagged, causal): mortgage rate (L1), migration (L2), consents (L4), filled jobs (L1)
  - Additional variables (OCR, listings, total stock, sales, days-to-sell, median price) are screened and reported
    in the Appendix, but are NOT added to the main model (avoid leakage/post-hoc feature creep).
  - We keep “total_stock_akl_imp_lag1” as a robustness variant in Appendix only.

### Outputs (saved in OUT_DIR):
  - panel_summary.csv
  - monthly_panel.csv
  - monthly_panel_imputed_causal.csv
  - imputation_report_causal.csv
  - cv_metrics_log_nonseasonal.csv
  - cv_metrics_log_seasonal.csv
  - cv_champions_by_horizon.csv
  - cv_allmodels_logrmse.csv
  - test_predictions_laststep.csv
  - final_test_champions_by_horizon.csv
  - test_dm_winner_vs_snaive.csv
  - test_metrics_winners_level.csv
  - test_metrics_level_allmodels.csv
  - test_metrics_log_allmodels.csv
  - plots_test/test_actual_vs_winner_h{1,3,6,12}.png
  - appendix_leadlag_corr.csv
  - appendix_oos_singlevar_rmse.csv

### Step 0: Main CSV File Creation
- Compiling all raw csv data files to one main output csv file
- Dropping duplicates for dates

In [1]:
import os, math, warnings
import numpy as np
import pandas as pd
BASE = os.path.abspath(os.getcwd())
DATA = r"C:\AUT_Lecture\Semester-2\ENGE817_STEM_Research_Method\Data_Collection\Extracted_Data_from_Original_Data_for_Analysis"
OUT  = os.path.join(os.path.dirname(DATA), "output")  # C:\...\Data_Collection\output
os.makedirs(OUT, exist_ok=True)

def _read_csv(name, parse_dates=True):
    path = os.path.join(DATA, name)
    if not os.path.exists(path):
        print(f"WARNING: {name} not found, skipping.")
        return None
    df = pd.read_csv(path)
    if parse_dates and "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], utc=False)
        # force month-end for alignment
        df["date"] = df["date"] + pd.offsets.MonthEnd(0)
    return df

def _merge(left, right):
    if left is None:
        return right
    if right is None:
        return left
    return pd.merge(left, right, on="date", how="outer")

def build_panel():
    dfs = []
    dfs.append(_read_csv("reinz_auckland_hpi.csv"))
    dfs.append(_read_csv("rbnz_mortgage_rates_b20.csv"))
    dfs.append(_read_csv("offical_cash_rate_(ocr).csv"))
    dfs.append(_read_csv("stats_nz_building_consents_auckland.csv"))
    dfs.append(_read_csv("stats_nz_net_migration_nz.csv"))
    dfs.append(_read_csv("stats_nz_mei_filled_jobs_nz.csv"))
    dfs.append(_read_csv("realestate_nz_listings_auckland.csv"))
    dfs.append(_read_csv("reinz_auckland_market_activity.csv"))

    panel = None
    for d in dfs:
        panel = _merge(panel, d)

    # If nothing loaded
    if panel is None or panel.empty:
        raise RuntimeError("No data loaded. Check filenames/columns in DATA folder.")

    # Sort and drop duplicates
    panel = panel.sort_values("date").drop_duplicates("date")

    # Ensure monthly frequency index (use 'ME' = month end)
    panel = panel.set_index("date").asfreq("ME")

    # Target transformations
    if "hpi_index" in panel.columns:
        panel["y_log"] = np.log(panel["hpi_index"])
        panel["y_diff"] = panel["y_log"].diff()
    else:
        print("WARNING: hpi_index not found. Please populate reinz_auckland_hpi.csv")

    # Create lags for market activity (avoid look-ahead)
    for col in ["sales_count_akl", "days_to_sell_akl"]:
        if col in panel.columns:
            for L in [1, 2, 3]:
                panel[f"{col}_lag{L}"] = panel[col].shift(L)

    # Simple sanity outputs
    summary = panel.describe(include="all")
    summary.to_csv(os.path.join(OUT, "panel_summary.csv"))
    out_path = os.path.join(OUT, "monthly_panel.csv")
    panel.to_csv(out_path)
    print(f"Wrote: {out_path}")
    return panel

if __name__ == "__main__":
    print("BASE:", BASE)
    print("DATA:", DATA)
    print("DATA exists:", os.path.exists(DATA))
    if os.path.exists(DATA):
        print("DATA files:", os.listdir(DATA))  
    build_panel()

BASE: C:\Users\icora\AUT Lab Learning
DATA: C:\AUT_Lecture\Semester-2\ENGE817_STEM_Research_Method\Data_Collection\Extracted_Data_from_Original_Data_for_Analysis
DATA exists: True
DATA files: ['offical_cash_rate_(ocr).csv', 'rbnz_mortgage_rates_b20.csv', 'realestate_nz_listings_auckland.csv', 'reinz_auckland_hpi.csv', 'reinz_auckland_market_activity.csv', 'stats_nz_building_consents_auckland.csv', 'stats_nz_mei_filled_jobs_nz.csv', 'stats_nz_net_migration_nz.csv']
Wrote: C:\AUT_Lecture\Semester-2\ENGE817_STEM_Research_Method\Data_Collection\output\monthly_panel.csv


### Step 1: Imports & Configuration

In [2]:
# ============================== 1) Imports & Config =============================
import os, math, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.tseries.offsets import DateOffset, MonthEnd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX

warnings.filterwarnings("ignore")
pd.set_option("display.float_format", lambda v: f"{v:,.6f}")

# ---- Paths (edit OUT_DIR only) ----
OUT_DIR = r"C:\AUT_Lecture\Semester-2\ENGE817_STEM_Research_Method\Data_Collection\output"
PANEL_PATH = os.path.join(OUT_DIR, "monthly_panel.csv")

# ---- Timeline windows (month-end timestamps) ----
TRAIN_END  = pd.Timestamp("2024-08-31")  # CV window ends by origin
TEST_START = pd.Timestamp("2024-09-30")  # test window by TARGET month (month-end)
TEST_END   = pd.Timestamp("2025-08-31")

# ---- Evaluation settings ----
MIN_TRAIN    = 24
HORIZONS     = (1, 3, 6, 12)
ARIMA_ORDERS = [(1,1,0), (0,1,1), (1,1,1)]
TARGET       = "y_log_imp"

### Step 2: Causal Imputation (in-memory)

In [3]:
# ====================== 2) Causal Imputation (in-memory) =======================

def _seasonal_fill_causal(s: pd.Series):
    """Same-month-last-year → month-of-year median (past only) → forward-fill."""
    s = s.copy().asfreq("M")
    idx = s.index
    miss = s.isna()
    if not miss.any(): return s
    # 1) same month last year
    for i in np.where(miss)[0]:
        t = idx[i]; ly = t - pd.DateOffset(years=1)
        if ly in idx and pd.notna(s.loc[ly]):
            s.iloc[i] = s.loc[ly]
    # 2) month-of-year median from past
    if s.isna().any():
        moy = s.index.month
        for i in np.where(s.isna())[0]:
            m = moy[i]
            hist = s[(moy==m) & (s.index < s.index[i])].dropna()
            if len(hist) >= 3:
                s.iloc[i] = hist.median()
    # 3) causal fallback
    return s.ffill()

def _interp_fill_causal(s: pd.Series, nonneg=False):
    x = s.asfreq("M").ffill()
    return x.clip(lower=0) if nonneg else x

def build_imputed_panel(df_raw: pd.DataFrame) -> pd.DataFrame:
    """Create *_imp, *_miss, and y_* transforms causally. No disk writes here."""
    df = df_raw.copy().asfreq("M")

    # Target (HPI)
    if "hpi_index" in df.columns:
        df["hpi_index_imp"] = _interp_fill_causal(df["hpi_index"])
        df["y_log_imp"]     = np.log(df["hpi_index_imp"])
        df["y_diff_imp"]    = df["y_log_imp"].diff()

    # Rates / OCR (clip 0..25)
    for c in ["mort_rate_1y", "mort_rate_float", "ocr"]:
        if c in df.columns:
            df[f"{c}_miss"] = df[c].isna().astype(int)
            df[f"{c}_imp"]  = _interp_fill_causal(df[c], nonneg=True).clip(0, 25)

    # Seasonal counts (+round)
    for c in ["dwellings_consent_count","new_listings_akl","total_stock_akl",
              "sales_count_akl","days_to_sell_akl","median_price_akl"]:
        if c in df.columns:
            df[f"{c}_miss"] = df[c].isna().astype(int)
            df[f"{c}_imp"]  = np.rint(_seasonal_fill_causal(df[c]).clip(lower=0)).astype(float)

    # Smooth activity
    if "filled_jobs_nz" in df.columns:
        df["filled_jobs_nz_miss"] = df["filled_jobs_nz"].isna().astype(int)
        df["filled_jobs_nz_imp"]  = np.rint(_interp_fill_causal(df["filled_jobs_nz"], nonneg=True)).astype(float)

    if "net_migration_nz" in df.columns:
        df["net_migration_nz_miss"] = df["net_migration_nz"].isna().astype(int)
        df["net_migration_nz_imp"]  = _interp_fill_causal(df["net_migration_nz"], nonneg=False)

    return df

### Step 3: Features (Exogenous Regressors)

In [4]:
# ========================== 3) Features =============================

def add_lags(df, col, lags):
    for L in lags:
        df[f"{col}_lag{L}"] = df[col].shift(L)
    return df

def build_exog(df):
    """
    Main ARIMAX drivers only (pre-registered, causal lags):
      - mort_rate_1y_imp (L1), net_migration_nz_imp (L2),
        dwellings_consent_count_imp (L4), filled_jobs_nz_imp (L1)
    Also include lagged *_miss flags to signal imputed periods.
    """
    exog_lags = {
        "mort_rate_1y_imp": [1],
        "net_migration_nz_imp": [2],
        "dwellings_consent_count_imp": [4],
        "filled_jobs_nz_imp": [1],
    }
    EXO = []
    for c, lags in exog_lags.items():
        if c in df.columns:
            add_lags(df, c, lags)
            EXO += [f"{c}_lag{L}" for L in lags]

    for base in ["mort_rate_1y_miss","net_migration_nz_miss",
                 "dwellings_consent_count_miss","filled_jobs_nz_miss"]:
        if base in df.columns:
            add_lags(df, base, [1])
            EXO.append(f"{base}_lag1")

    return df, EXO

### Step 4: Models - ETS & (S)ARIMA / ARIMAX fitters

In [5]:
# ================ 4) Models: ETS & (S)ARIMA / ARIMAX fitters ===================

def fit_ets(y: pd.Series):
    """ETS A/A if long enough; otherwise simple level."""
    if len(y) < 24:
        return ExponentialSmoothing(y, trend=None, seasonal=None).fit()
    return ExponentialSmoothing(y, trend="add", seasonal="add", seasonal_periods=12).fit()

def fit_sarimax(y, X, order, seasonal_flag):
    seas = (0,0,0,0) if not seasonal_flag else (0,1,1,12)
    return SARIMAX(y, exog=X, order=order, seasonal_order=seas,
                   enforce_stationarity=False, enforce_invertibility=False
                  ).fit(disp=False, maxiter=200)

### Step 5: Metrics & Tests

In [6]:
# ============================= 5) Metrics & Tests ==============================

def metrics_table(df, label="log"):
    out=[]
    for h, g in df.groupby("h"):
        y = g["y_true"].values
        def metr(col):
            e = y - g[col].values
            return dict(MAE=float(np.mean(np.abs(e))),
                        RMSE=float(np.sqrt(np.mean(e**2))),
                        MAPE=float(np.mean(np.abs(e/np.where(np.abs(y)<1e-12, np.nan, y)))*100))
        row = {"h": int(h), "scale": label}
        for m in ["snaive","ets","arima","arimax"]:
            if m in g.columns:
                row.update({f"{m}_{k}": v for k,v in metr(m).items()})
        out.append(row)
    return pd.DataFrame(out).sort_values("h")

def _normal_cdf(x): return 0.5*(1.0+math.erf(x/math.sqrt(2.0)))

def dm_test(e_model, e_base, h):
    """
    Diebold–Mariano test on squared errors; small-sample lag cap L = min(h-1, T//4, >=1).
    Returns (stat, p).
    """
    e1, e2 = np.asarray(e_model), np.asarray(e_base)
    d = (e1**2 - e2**2); dbar = np.mean(d); T = len(d)
    if T < 3: return np.nan, np.nan
    L = min(max(h-1,1), max(1, T//4))
    gamma0 = np.var(d, ddof=1)
    cov = 0.0
    for l in range(1, L+1):
        wl = 1 - l/(L+1)
        cov += 2*wl*np.cov(d[l:], d[:-l], ddof=1)[0,1]
    var = (gamma0 + cov)/T
    if not np.isfinite(var) or var<=0: return np.nan, np.nan
    stat = dbar/np.sqrt(var); p = 2*(1-_normal_cdf(abs(stat)))
    return float(stat), float(p)

### Step 6: Rolling-origin cross-validation (model selection)

In [7]:
# ========================== 6) Rolling-origin CV ===============================

def evaluate_cv(df_base, seasonal_flag, target=TARGET,
                horizons=HORIZONS, arima_orders=ARIMA_ORDERS, min_train=MIN_TRAIN):
    df = df_base.copy()
    df, EXO = build_exog(df)
    df = df.loc[:TRAIN_END].dropna(subset=[target] + EXO)
    dates = df.index

    rows=[]
    for t_idx in range(max(min_train, 0), len(dates)-1):
        origin = dates[t_idx]
        y_tr   = df.loc[:origin, target]
        X_tr   = df.loc[:origin, EXO] if EXO else None

        # ETS fit
        ets_fit = None
        try: ets_fit = fit_ets(y_tr)
        except Exception: pass

        # Best ARIMA by AIC
        best_arima, best_aic = None, np.inf
        for od in arima_orders:
            try:
                m = fit_sarimax(y_tr, None, od, seasonal_flag)
                if m.aic < best_aic: best_arima, best_aic = m, m.aic
            except Exception: pass

        # Best ARIMAX by AIC
        best_arimax, best_aicx = None, np.inf
        if X_tr is not None:
            for od in arima_orders:
                try:
                    mx = fit_sarimax(y_tr, X_tr, od, seasonal_flag)
                    if mx.aic < best_aicx: best_arimax, best_aicx = mx, mx.aic
                except Exception: pass

        # Forecast horizons
        for h in horizons:
            if t_idx + h >= len(dates): continue
            fut_idx = dates[t_idx+1 : t_idx+1+h]
            X_fut   = df.loc[fut_idx, EXO] if EXO else None

            y_true_last = df.loc[fut_idx[-1], target]
            snaive_last = (np.repeat(y_tr.iloc[-12], h)[-1] if len(y_tr)>=12 else y_tr.iloc[-1])

            ets_last = np.nan
            if ets_fit is not None:
                try: ets_last = ets_fit.forecast(h).iloc[-1]
                except Exception: pass

            arima_last = np.nan
            if best_arima is not None:
                try: arima_last = best_arima.forecast(h).iloc[-1]
                except Exception: pass

            arimax_last = np.nan
            if best_arimax is not None and X_fut is not None:
                try: arimax_last = best_arimax.forecast(h, exog=X_fut).iloc[-1]
                except Exception: pass

            rows.append({"t_origin": origin, "h": h, "y_true": y_true_last,
                         "snaive": snaive_last, "ets": ets_last,
                         "arima": arima_last, "arimax": arimax_last})

    results = pd.DataFrame(rows)
    metrics_log = metrics_table(results, "log")

    # DM vs snaive on CV (optional, not saved here to keep files lean)
    dm_rows=[]
    for h, g in results.groupby("h"):
        base_e = g["y_true"] - g["snaive"]
        for m in ["ets","arima","arimax"]:
            if m not in g.columns or g[m].isna().any():
                stat, p = np.nan, np.nan
            else:
                e = g["y_true"] - g[m]
                stat, p = dm_test(e.values, base_e.values, h=int(h))
            dm_rows.append({"h": int(h), "model": m, "DM_stat_vs_snaive": stat, "p_value": p})
    dm = pd.DataFrame(dm_rows).sort_values(["h","model"])
    return results, metrics_log, dm

### Step 7: Test evaluation (champions only)

In [8]:
# ================== 7) Test evaluation (champions only) ========================

def evaluate_test(df_base, winners: dict, target=TARGET,
                  horizons=HORIZONS, arima_orders=ARIMA_ORDERS, min_train=MIN_TRAIN):
    df = df_base.copy()
    df, EXO = build_exog(df)
    df = df.dropna(subset=[target] + EXO)
    dates = df.index

    rows=[]
    for h in horizons:
        win = winners.get(h, {"model":"ets","seasonal":False})
        mtype, seas = win["model"], win["seasonal"]

        for i in range(len(dates)-h):
            origin = dates[i]
            target_date = (origin.to_period("M") + h).to_timestamp("M")
            if not (TEST_START <= target_date <= TEST_END): continue
            if i < min_train: continue

            y_tr = df.loc[:origin, target]
            X_tr = df.loc[:origin, EXO] if EXO else None
            fut_idx = dates[i+1:i+1+h]
            X_fut   = df.loc[fut_idx, EXO] if EXO else None

            snaive_last = (np.repeat(y_tr.iloc[-12], h)[-1] if len(y_tr)>=12 else y_tr.iloc[-1])
            y_true_last = df.loc[fut_idx[-1], target]

            ets_last = arima_last = arimax_last = np.nan
            if mtype == "ets":
                try: ets_last = fit_ets(y_tr).forecast(h).iloc[-1]
                except Exception: pass
            elif mtype == "arima":
                best, aic = None, np.inf
                for od in arima_orders:
                    try:
                        m = fit_sarimax(y_tr, None, od, seas)
                        if m.aic < aic: best, aic = m, m.aic
                    except Exception: pass
                if best is not None:
                    try: arima_last = best.forecast(h).iloc[-1]
                    except Exception: pass
            elif mtype == "arimax":
                best, aic = None, np.inf
                for od in arima_orders:
                    try:
                        mx = fit_sarimax(y_tr, X_tr, od, seas)
                        if mx.aic < aic: best, aic = mx, mx.aic
                    except Exception: pass
                if best is not None and X_fut is not None:
                    try: arimax_last = best.forecast(h, exog=X_fut).iloc[-1]
                    except Exception: pass

            rows.append({"t_origin": origin, "h": h, "target_date": target_date,
                         "y_true": y_true_last, "snaive": snaive_last,
                         "ets": ets_last, "arima": arima_last, "arimax": arimax_last,
                         "winner_model": mtype, "winner_seasonal": seas})
    return pd.DataFrame(rows).sort_values(["h","target_date"])

### Step 8: Additional exogenous variables screening (Appendix)

In [9]:
# ===================== 8) Appendix — exogenous screening =======================

def appendix_exogenous_quickcheck(df: pd.DataFrame, out_dir: str):
    """
    Produces two Appendix CSVs:
      - appendix_leadlag_corr.csv        (lead–lag with Δlog(HPI))
      - appendix_oos_singlevar_rmse.csv  (tiny OOS Δlog(HPI) ~ lagged x check)
    This does NOT change the main model.
    """
    ser_map = [
        ("OCR",                  "ocr_imp"),
        ("New listings (AKL)",   "new_listings_akl_imp"),
        ("Total stock (AKL)",    "total_stock_akl_imp"),
        ("Sales count (AKL)",    "sales_count_akl_imp"),
        ("Days to sell (AKL)",   "days_to_sell_akl_imp"),
        ("Median price (AKL)",   "median_price_akl_imp"),
    ]

    # Δlog(HPI)
    y = df["y_log_imp"].diff()
    leadlag_rows=[]
    for label, col in ser_map:
        if col not in df.columns: continue
        x0 = pd.Series(df[col], index=df.index)
        lags = range(0, 7)
        corr = {}
        for L in lags:
            xx = x0.shift(L).reindex(y.index)
            c  = np.corrcoef(y.dropna(), xx.loc[y.dropna().index].dropna())[0,1] if xx.notna().sum()>24 else np.nan
            corr[f"lag{L}"] = c
        # pick best absolute
        best_L = int(max(lags, key=lambda L: abs(corr.get(f"lag{L}", np.nan)) if pd.notna(corr.get(f"lag{L}", np.nan)) else -1))
        leadlag_rows.append({
            "series": label, "col": col, **corr,
            "best_lag": best_L,
            "best_corr_signed": corr.get(f"lag{best_L}", np.nan),
            "best_abs_corr": abs(corr.get(f"lag{best_L}", np.nan)) if pd.notna(corr.get(f"lag{best_L}", np.nan)) else np.nan
        })
    pd.DataFrame(leadlag_rows).to_csv(os.path.join(out_dir, "appendix_leadlag_corr.csv"), index=False)

    # Tiny OOS check on Δlog(HPI) ~ lagged x  (pseudo-regression via one-step naive vs x-lag)
    oos_rows=[]
    for label, col in ser_map:
        if col not in df.columns: continue
        best_lag = int(pd.read_csv(os.path.join(out_dir, "appendix_leadlag_corr.csv"))
                       .query("col == @col")["best_lag"].iloc[0])
        y2 = y.copy()
        xL = df[col].shift(best_lag)
        # align
        z = pd.concat([y2, xL], axis=1, keys=["dy", "x"]).dropna()
        if len(z) < 36:
            oos_rows.append({"series": label, "col": col, "chosen_causal_lag": best_lag,
                             "RMSE_model": np.nan, "RMSE_naive0": np.nan,
                             "%_improvement_vs_naive0": np.nan, "n_test_obs": len(z)})
            continue
        # split 70/30
        cut = int(len(z)*0.7)
        tr, te = z.iloc[:cut], z.iloc[cut:]
        # simple linear fit dy ~ x on train
        xtr = tr["x"].values; ytr = tr["dy"].values
        beta = np.dot(xtr, ytr) / np.dot(xtr, xtr) if np.dot(xtr, xtr) != 0 else 0.0
        yhat = te["x"].values * beta
        rmse_model = float(np.sqrt(np.mean((te["dy"].values - yhat)**2)))
        rmse_naive = float(np.sqrt(np.mean((te["dy"].values - 0.0)**2)))  # zero-change baseline on dy
        gain = 100.0 * (rmse_naive - rmse_model) / rmse_naive if rmse_naive>0 else np.nan

        oos_rows.append({"series": label, "col": col, "chosen_causal_lag": best_lag,
                         "RMSE_model": rmse_model, "RMSE_naive0": rmse_naive,
                         "%_improvement_vs_naive0": gain, "n_test_obs": len(te)})
    pd.DataFrame(oos_rows).to_csv(os.path.join(out_dir, "appendix_oos_singlevar_rmse.csv"), index=False)


In [10]:
# === Appendix A1: lead–lag correlation matrix + heatmap ===
import os, numpy as np, pandas as pd
import matplotlib.pyplot as plt

OUT_DIR = r"C:\AUT_Lecture\Semester-2\ENGE817_STEM_Research_Method\Data_Collection\output"
src = os.path.join(OUT_DIR, "appendix_leadlag_corr.csv")
fig_dir = os.path.join(OUT_DIR, "appendix_figs"); os.makedirs(fig_dir, exist_ok=True)

# 1) Load and pivot to matrix (rows=series, cols=lag0..lag6)
df = pd.read_csv(src)
# Use the human-readable 'series' as row labels, fall back to 'col' if needed
row_label = "series" if "series" in df.columns else "col"
lag_cols = [c for c in df.columns if c.startswith("lag")]
mat = (df[[row_label] + lag_cols]
       .set_index(row_label)
       .sort_index())

# Optional: round for a clean table in the appendix
mat_rounded = mat.round(3)
mat_path = os.path.join(OUT_DIR, "appendix_A1_leadlag_corr_matrix.csv")
mat_rounded.to_csv(mat_path, index=True)
print("Saved table:", mat_path)

# 2) Heatmap (matplotlib only)
plt.figure(figsize=(10, max(3, 0.5*len(mat))))
im = plt.imshow(mat.values, aspect='auto', interpolation='nearest')
plt.colorbar(im, fraction=0.046, pad=0.04, label="Correlation with Δlog(HPI)")
plt.xticks(ticks=np.arange(len(lag_cols)), labels=lag_cols, rotation=0)
plt.yticks(ticks=np.arange(len(mat.index)), labels=mat.index)
plt.title("Lead–lag correlations with Δlog(HPI) (positive = co-move)")
plt.tight_layout()
fig_path = os.path.join(fig_dir, "Fig_A1_leadlag_corr_heatmap.png")
plt.savefig(fig_path, dpi=200)
plt.close()
print("Saved figure:", fig_path)

# 3) (Optional) annotate a version with numbers for review (not for print if busy)
plt.figure(figsize=(10, max(3, 0.5*len(mat))))
im = plt.imshow(mat.values, aspect='auto', interpolation='nearest')
plt.colorbar(im, fraction=0.046, pad=0.04, label="Correlation")
plt.xticks(ticks=np.arange(len(lag_cols)), labels=lag_cols, rotation=0)
plt.yticks(ticks=np.arange(len(mat.index)), labels=mat.index)
# add text annotations
vals = mat.round(2).values
for i in range(vals.shape[0]):
    for j in range(vals.shape[1]):
        plt.text(j, i, f"{vals[i,j]:.2f}",
                 ha="center", va="center", fontsize=8)
plt.title("Lead–lag correlations (annotated)")
plt.tight_layout()
fig_path2 = os.path.join(fig_dir, "Fig_A1b_leadlag_corr_heatmap_annotated.png")
plt.savefig(fig_path2, dpi=200)
plt.close()
print("Saved figure:", fig_path2)


Saved table: C:\AUT_Lecture\Semester-2\ENGE817_STEM_Research_Method\Data_Collection\output\appendix_A1_leadlag_corr_matrix.csv
Saved figure: C:\AUT_Lecture\Semester-2\ENGE817_STEM_Research_Method\Data_Collection\output\appendix_figs\Fig_A1_leadlag_corr_heatmap.png
Saved figure: C:\AUT_Lecture\Semester-2\ENGE817_STEM_Research_Method\Data_Collection\output\appendix_figs\Fig_A1b_leadlag_corr_heatmap_annotated.png


### Step 9: Main Analysis Parts & Printing Results

In [11]:
# ================================ 9) MAIN ======================================

def main():
    # --- Load main assembled panel ---
    if not os.path.exists(PANEL_PATH):
        raise FileNotFoundError(f"monthly_panel.csv not found at: {PANEL_PATH}")
    raw = (pd.read_csv(PANEL_PATH, parse_dates=["date"])
             .set_index("date")
             .asfreq("M"))

#### Step 9.1: Printing causal imputation (in-memory) + save the exact panel used (audit)

In [12]:
# ===== load panel + causal imputation + save audit files (standalone cell) =====

# Load main assembled panel
if not os.path.exists(PANEL_PATH):
    raise FileNotFoundError(f"monthly_panel.csv not found at: {PANEL_PATH}")

raw = (pd.read_csv(PANEL_PATH, parse_dates=["date"])
         .set_index("date")
         .asfreq("M"))

# Causal imputation (in-memory) + save the exact panel used
data = build_imputed_panel(raw)
os.makedirs(OUT_DIR, exist_ok=True)
data.to_csv(os.path.join(OUT_DIR, "monthly_panel_imputed_causal.csv"),
            index=True, index_label="date")
print("Saved:", os.path.join(OUT_DIR, "monthly_panel_imputed_causal.csv"))

# Quick imputation report
rep = []
for base in ["hpi_index","mort_rate_1y","mort_rate_float","ocr",
             "dwellings_consent_count","new_listings_akl","total_stock_akl",
             "sales_count_akl","days_to_sell_akl","filled_jobs_nz","net_migration_nz"]:
    miss_col, imp_col = f"{base}_miss", f"{base}_imp"
    if (miss_col in data.columns) or (imp_col in data.columns):
        rep.append({
            "series": base,
            "n_rows": len(data),
            "n_missing_flag": int(data[miss_col].sum()) if miss_col in data.columns else None,
            "has_imp_col": imp_col in data.columns
        })

pd.DataFrame(rep).to_csv(os.path.join(OUT_DIR, "imputation_report_causal.csv"), index=False)
print("Saved:", os.path.join(OUT_DIR, "imputation_report_causal.csv"))


Saved: C:\AUT_Lecture\Semester-2\ENGE817_STEM_Research_Method\Data_Collection\output\monthly_panel_imputed_causal.csv
Saved: C:\AUT_Lecture\Semester-2\ENGE817_STEM_Research_Method\Data_Collection\output\imputation_report_causal.csv


#### Step 9.2: Cross-validation (non-seasonal & seasonal flags)

In [14]:
# --- Cross-validation (non-seasonal & seasonal flags) ---
cv_non_res, cv_non_met, _ = evaluate_cv(data, seasonal_flag=False, target=TARGET)
cv_sea_res, cv_sea_met, _  = evaluate_cv(data, seasonal_flag=True,  target=TARGET)

cv_non_met.to_csv(os.path.join(OUT_DIR, "cv_metrics_log_nonseasonal.csv"), index=False)
cv_sea_met.to_csv(os.path.join(OUT_DIR, "cv_metrics_log_seasonal.csv"), index=False)

# Merge to compare seasonal vs non-seasonal
keep = ["h","snaive_RMSE","ets_RMSE","arima_RMSE","arimax_RMSE"]
A = cv_non_met[keep].rename(columns=lambda c: c if c=="h" else f"non_{c}")
B = cv_sea_met[keep].rename(columns=lambda c: c if c=="h" else f"seas_{c}")
M = A.merge(B, on="h", how="inner")

# (NEW) Long table with ALL models per horizon
all_rows = []
winners, winner_rows = {}, []
for _, r in M.iterrows():
    h = int(r["h"])
    candidates = {
        ("snaive", False): r["non_snaive_RMSE"],
        ("ets",    False): r["non_ets_RMSE"],
        ("arima",  False): r["non_arima_RMSE"],
        ("arimax", False): r["non_arimax_RMSE"],
        ("arima",  True ): r["seas_arima_RMSE"],
        ("arimax", True ): r["seas_arimax_RMSE"],
    }
    # Write all candidates
    for (m, seas), rmse in candidates.items():
        all_rows.append({
            "h": h, "model": m, "seasonal": bool(seas), "RMSE_log": float(rmse)
        })
    # Winner
    best_key = min(candidates, key=lambda k: candidates[k])
    winners[h] = {"model": best_key[0], "seasonal": best_key[1],
                  "cv_rmse_log": float(candidates[best_key])}
    winner_rows.append({"h": h, "winner_model": best_key[0],
                        "winner_seasonal": best_key[1],
                        "winner_RMSE_log": float(candidates[best_key])})

cv_all = pd.DataFrame(all_rows).sort_values(["h","model","seasonal"])
cv_all.to_csv(os.path.join(OUT_DIR, "cv_allmodels_logrmse.csv"), index=False)

champ_cv = pd.DataFrame(winner_rows).sort_values("h")
champ_cv.to_csv(os.path.join(OUT_DIR, "cv_champions_by_horizon.csv"), index=False)

print("\n=== CV Champions (2018-01 → 2024-08) ===")
print(champ_cv.to_string(index=False))
print("\nSaved:", os.path.join(OUT_DIR, "cv_allmodels_logrmse.csv"))



=== CV Champions (2018-01 → 2024-08) ===
 h winner_model  winner_seasonal  winner_RMSE_log
 1        arima            False         0.015591
 3        arima            False         0.035771
 6        arima            False         0.064802
12       arimax            False         0.097382

Saved: C:\AUT_Lecture\Semester-2\ENGE817_STEM_Research_Method\Data_Collection\output\cv_allmodels_logrmse.csv


#### Step 9.3: Test evaluation (target months 2024-09 → 2025-08)

In [15]:
# --- Test evaluation (target months 2024-09 → 2025-08) ---
test = evaluate_test(data, winners, target=TARGET)
test = test[(test["target_date"] >= TEST_START) & (test["target_date"] <= TEST_END)]
test.to_csv(os.path.join(OUT_DIR, "test_predictions_laststep.csv"), index=False)

def metr(g, col):
    y = g["y_true"].to_numpy(); yhat = g[col].to_numpy()
    e = y - yhat
    return dict(MAE_log=float(np.mean(np.abs(e))),
                RMSE_log=float(np.sqrt(np.mean(e**2))),
                MAPE_log=float(np.mean(np.abs(e/np.where(np.abs(y)<1e-12, np.nan, y)))*100))

# (NEW) all models on TEST — log metrics
log_rows_all, rows_winner = [], []
for h, g in test.groupby("h"):
    models = [m for m in ["snaive","ets","arima","arimax"]
              if m in g.columns and g[m].notna().any()]
    stats = {m: metr(g, m) for m in models}
    # write all
    for m in models:
        log_rows_all.append({"h": int(h), "model": m, **stats[m]})
    # pick winner
    winner = min(stats, key=lambda m: stats[m]["RMSE_log"])
    rows_winner.append({"h": int(h), "winner_test": winner, **stats[winner],
                        "snaive_RMSE_log": stats.get("snaive",{}).get("RMSE_log", np.nan),
                        "ets_RMSE_log":    stats.get("ets",{}).get("RMSE_log", np.nan),
                        "arima_RMSE_log":  stats.get("arima",{}).get("RMSE_log", np.nan),
                        "arimax_RMSE_log": stats.get("arimax",{}).get("RMSE_log", np.nan)})

test_log_all = pd.DataFrame(log_rows_all).sort_values(["h","model"])
test_log_all.to_csv(os.path.join(OUT_DIR, "test_metrics_log_allmodels.csv"), index=False)

champ_test = pd.DataFrame(rows_winner).sort_values("h")
champ_test.to_csv(os.path.join(OUT_DIR, "final_test_champions_by_horizon.csv"), index=False)

print("\n=== TEST Champions (target 2024-09 → 2025-08) ===")
print(champ_test.to_string(index=False))
print("Saved:", os.path.join(OUT_DIR, "test_metrics_log_allmodels.csv"))



=== TEST Champions (target 2024-09 → 2025-08) ===
 h winner_test  MAE_log  RMSE_log  MAPE_log  snaive_RMSE_log  ets_RMSE_log  arima_RMSE_log  arimax_RMSE_log
 1       arima 0.009719  0.010778  0.119761         0.017668           NaN        0.010778              NaN
 3       arima 0.016250  0.019891  0.200257         0.023272           NaN        0.019891              NaN
 6       arima 0.020351  0.025501  0.250913         0.032773           NaN        0.025501              NaN
12      snaive 0.021214  0.027841  0.261549         0.027841           NaN             NaN         0.074024
Saved: C:\AUT_Lecture\Semester-2\ENGE817_STEM_Research_Method\Data_Collection\output\test_metrics_log_allmodels.csv


#### Step 9.4: DM test (winner vs seasonal-naive) on TEST (log scale)

In [16]:
    # --- DM test (winner vs seasonal-naive) on TEST (log scale) ---
    dm_rows=[]
    for h, g in test.groupby("h"):
        y = g["y_true"].to_numpy()
        s = g["snaive"].to_numpy()
        win_name = champ_test.loc[champ_test["h"]==h, "winner_test"].values[0]
        w = g[win_name].to_numpy()
        if np.allclose(w, s, equal_nan=False):
            stat, p = 0.0, 1.0
        else:
            stat, p = dm_test(y - w, y - s, h=int(h))
        dm_rows.append({"h": int(h), "winner_model": win_name,
                        "DM_stat_winner_vs_snaive": stat, "p_value": p})
    dm_test_win = pd.DataFrame(dm_rows).sort_values("h")
    dm_test_win.to_csv(os.path.join(OUT_DIR, "test_dm_winner_vs_snaive.csv"), index=False)

    print("\n=== DM (winner vs seasonal-naïve, log) on TEST ===")
    print(dm_test_win.to_string(index=False))



=== DM (winner vs seasonal-naïve, log) on TEST ===
 h winner_model  DM_stat_winner_vs_snaive  p_value
 1        arima                 -1.675568 0.093823
 3        arima                 -0.538923 0.589940
 6        arima                 -1.046331 0.295408
12       snaive                  0.000000 1.000000


#### Step 9.5: Level metrics for winners (intuitive HPI units)

In [20]:
# --- Level metrics for ALL models (HPI units) ---
level_rows_all = []
for h, g in test.groupby("h"):
    y_lvl = np.exp(g["y_true"].to_numpy())
    for m in [c for c in ["snaive","ets","arima","arimax"] if c in g.columns and g[c].notna().any()]:
        w_lvl = np.exp(g[m].to_numpy())
        e = y_lvl - w_lvl
        level_rows_all.append({
            "h": int(h), "model": m,
            "MAE_level": float(np.mean(np.abs(e))),
            "RMSE_level": float(np.sqrt(np.mean(e**2))),
            "MAPE_level": float(np.mean(np.abs(e/np.where(y_lvl==0,np.nan,y_lvl)))*100)
        })

test_level_all = pd.DataFrame(level_rows_all).sort_values(["h","model"])
test_level_all.to_csv(os.path.join(OUT_DIR, "test_metrics_level_allmodels.csv"), index=False)

print("\nSaved:", os.path.join(OUT_DIR, "test_metrics_level_allmodels.csv"))
print("\nLevel metrics (RMSE_level in HPI units):")
print(test_level_all.to_string(index=False))



Saved: C:\AUT_Lecture\Semester-2\ENGE817_STEM_Research_Method\Data_Collection\output\test_metrics_level_allmodels.csv

Level metrics (RMSE_level in HPI units):
 h  model  MAE_level  RMSE_level  MAPE_level
 1  arima  32.558034   36.131299    0.973030
 1 snaive  49.416667   60.334484    1.468780
 3  arima  54.247804   66.253432    1.622499
 3 snaive  65.333333   79.015821    1.958306
 6  arima  67.958940   85.069473    2.040475
 6 snaive  93.916667  110.556547    2.825975
12 arimax 248.729413  259.377615    7.420794
12 snaive  71.583333   94.289890    2.149013


In [21]:
    # --- Level metrics for winners (intuitive HPI units) ---
    lvl_rows=[]
    for h, g in test.groupby("h"):
        win_name = champ_test.loc[champ_test["h"]==h, "winner_test"].values[0]
        y_lvl = np.exp(g["y_true"].to_numpy())
        w_lvl = np.exp(g[win_name].to_numpy())
        e = y_lvl - w_lvl
        lvl_rows.append({"h": int(h),
                         "MAE_level": float(np.mean(np.abs(e))),
                         "RMSE_level": float(np.sqrt(np.mean(e**2))),
                         "MAPE_level": float(np.mean(np.abs(e/np.where(y_lvl==0,np.nan,y_lvl)))*100)})
    test_metrics_level = pd.DataFrame(lvl_rows).sort_values("h")
    test_metrics_level.to_csv(os.path.join(OUT_DIR, "test_metrics_winners_level.csv"), index=False)

    print("\nLevel metrics (RMSE_level in HPI units):")
    print(test_metrics_level.to_string(index=False))


Level metrics (RMSE_level in HPI units):
 h  MAE_level  RMSE_level  MAPE_level
 1  32.558034   36.131299    0.973030
 3  54.247804   66.253432    1.622499
 6  67.958940   85.069473    2.040475
12  71.583333   94.289890    2.149013


#### Step 9.6: Plots: Actual vs Winner (level scale)

In [22]:
def appendix_exogenous_quickcheck(df, out_dir):
    """
    Save two appendix tables:
      1) Lead-lag correlations of Δlog(HPI) with candidate drivers (lags 0..6)
      2) Tiny OOS check: RMSE of one-step Δlog(HPI) using a single lagged driver vs naive(0)
    """
    y = df["y_log_imp"].diff().rename("dlog_hpi")  # Δlog(HPI)
    candidates = [
        ("OCR",                    "ocr_imp"),
        ("New listings (AKL)",     "new_listings_akl_imp"),
        ("Total stock (AKL)",      "total_stock_akl_imp"),
        ("Sales count (AKL)",      "sales_count_akl_imp"),
        ("Days to sell (AKL)",     "days_to_sell_akl_imp"),
        ("Median price (AKL)",     "median_price_akl_imp"),
    ]
    lags = [0,1,2,3,4,5,6]

    # ---------- A1) Lead–lag correlations (aligned inner join) ----------
    rows = []
    for label, col in candidates:
        if col not in df.columns:
            continue
        x0 = df[col]
        corr = {}
        for L in lags:
            xx = x0.shift(L)
            pair = pd.concat([y, xx], axis=1).dropna()   # inner align, drop any NaNs
            if len(pair) >= 24 and pair.iloc[:,1].std() > 0 and pair.iloc[:,0].std() > 0:
                c = pair.iloc[:,0].corr(pair.iloc[:,1])
            else:
                c = np.nan
            corr[f"lag{L}"] = c
        # pick best |corr|
        best_L, best_c = max(
            ((L, abs(corr[f"lag{L}"])) for L in lags if np.isfinite(corr[f"lag{L}"])),
            key=lambda t: t[1],
            default=(np.nan, np.nan)
        )
        best_signed = np.sign(corr.get(f"lag{best_L}", np.nan)) * best_c if np.isfinite(best_L) else np.nan
        rows.append({
            "series": label, "col": col, **corr,
            "best_lag": best_L,
            "best_corr_signed": best_signed,
            "best_abs_corr": best_c
        })

    a1 = pd.DataFrame(rows)
    a1.to_csv(os.path.join(out_dir, "appendix_leadlag_corr.csv"), index=False)

    # ---------- A2) Tiny OOS single-variable check ----------
    # Model: dlog_hpi_t = α + β * x_{t-L} + ε, rolling one-step, compare RMSE to naive0
    # We'll use the *causal* lag that had the best absolute corr but force L>=1
    rows = []
    for label, col in candidates:
        if col not in df.columns:
            continue
        # choose best causal lag (>=1); fallback to 1 if none
        sub = a1.loc[a1["col"] == col]
        if sub.empty:
            continue
        cand = [(int(L), abs(sub.iloc[0][f"lag{L}"])) for L in lags if L >= 1 and np.isfinite(sub.iloc[0][f"lag{L}"])]
        chosen_L = max(cand, key=lambda t: t[1])[0] if cand else 1

        # Build aligned frame for OOS
        xx = df[col].shift(chosen_L)
        frame = pd.concat([y, xx], axis=1).dropna()
        frame.columns = ["dlog_hpi", "xlag"]
        # Split train/test roughly at 2024-08 end (consistent with main analysis)
        split_date = pd.Timestamp("2024-08-31")
        tr = frame.loc[:split_date]
        te = frame.loc[split_date + pd.offsets.MonthEnd(1):]

        if len(tr) >= 24 and len(te) >= 6:
            # OLS on train
            Xtr = np.c_[np.ones(len(tr)), tr["xlag"].values]
            ytr = tr["dlog_hpi"].values
            try:
                beta = np.linalg.lstsq(Xtr, ytr, rcond=None)[0]
                # one-step predictions on test (static regression)
                Xte = np.c_[np.ones(len(te)), te["xlag"].values]
                yhat = Xte @ beta
                ytrue = te["dlog_hpi"].values
                rmse_model = float(np.sqrt(np.mean((ytrue - yhat)**2)))
                # Naive(0): predict zero change
                rmse_naive0 = float(np.sqrt(np.mean((ytrue - 0.0)**2)))
                rows.append({
                    "series": label,
                    "col": col,
                    "chosen_causal_lag": chosen_L,
                    "RMSE_model": rmse_model,
                    "RMSE_naive0": rmse_naive0,
                    "%_improvement_vs_naive0": 100.0*(rmse_naive0 - rmse_model)/rmse_naive0 if rmse_naive0>0 else np.nan,
                    "n_test_obs": int(len(te))
                })
            except Exception:
                pass

    a2 = pd.DataFrame(rows)
    a2.to_csv(os.path.join(out_dir, "appendix_oos_singlevar_rmse.csv"), index=False)


In [23]:
def main():
    # ... my existing steps that produce `test`, `champ_test`, and `data` ...

    # --- Plots: Actual vs Winner (level scale) ---
    plot_dir = os.path.join(OUT_DIR, "plots_test")
    os.makedirs(plot_dir, exist_ok=True)
    for h, g in test.groupby("h"):
        win_name = champ_test.loc[champ_test["h"] == h, "winner_test"].values[0]
        gg = g.sort_values("target_date")
        y_lvl, w_lvl = np.exp(gg["y_true"]), np.exp(gg[win_name])

        plt.figure(figsize=(8, 4.5))
        plt.plot(gg["target_date"], y_lvl, label="Actual")
        plt.plot(gg["target_date"], w_lvl, label=f"Winner (h={int(h)}: {win_name})")
        plt.title(f"Auckland HPI — Test window (h={int(h)})")
        plt.xlabel("Target month")
        plt.ylabel("HPI level")
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(plot_dir, f"test_actual_vs_winner_h{int(h)}.png"), dpi=160)
        plt.close()

    print(f"\nSaved plots to: {plot_dir}")

    # --- Appendix screening (does not alter main model) ---
    appendix_exogenous_quickcheck(data, OUT_DIR)
    print(
        "Saved Appendix tables:",
        os.path.join(OUT_DIR, "appendix_leadlag_corr.csv"),
        "and",
        os.path.join(OUT_DIR, "appendix_oos_singlevar_rmse.csv"),
    )


if __name__ == "__main__":
    main()



Saved plots to: C:\AUT_Lecture\Semester-2\ENGE817_STEM_Research_Method\Data_Collection\output\plots_test
Saved Appendix tables: C:\AUT_Lecture\Semester-2\ENGE817_STEM_Research_Method\Data_Collection\output\appendix_leadlag_corr.csv and C:\AUT_Lecture\Semester-2\ENGE817_STEM_Research_Method\Data_Collection\output\appendix_oos_singlevar_rmse.csv
